# Importo Google Playstore e lo metto in un dataframe

In [95]:
import pandas as pd
import numpy as np
import re

import pandas as pd
google=pd.read_csv('/Users/eliabettani/Desktop/FOUNDATIONS OF COMPUTER SCIENCE/googleplaystore.csv')
google.head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


In [96]:
# Leggo dataset googleplaystore_user_reviews.csv
playstore_review = pd.read_csv('googleplaystore_user_reviews.csv')
playstore_review.head(2)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462


# 1- Converto App sizes in numero

In [97]:
re_size = re.compile('^(?P<num>\d+\.?\d*)(?P<moltiplicatore>\w{0,1})$')

def size_tonumber(app_size):
    r = re_size.match(app_size)
    diz = {'M' : 1000000, 'k' : 1000}
    if r:
        molt = diz[r.group('moltiplicatore')]
        app_size = float(r.group('num'))*molt
    return(app_size)

google['Size'] = google['Size'].apply(size_tonumber)
google['Size'].head()

0    1.9e+07
1    1.4e+07
2    8.7e+06
3    2.5e+07
4    2.8e+06
Name: Size, dtype: object

# 2- Trasformo Installs in un numero

In [98]:
import re

# Ispezione possibili valori attributo Installs
print(google['Installs'].isnull().any()) # No missing
print(google.groupby('Installs').size())

# Presenza modalità 'Free'
google[google['Installs'] == 'Free']

False
Installs
0                    1
0+                  14
1+                  67
1,000+             907
1,000,000+        1579
1,000,000,000+      58
10+                386
10,000+           1054
10,000,000+       1252
100+               719
100,000+          1169
100,000,000+       409
5+                  82
5,000+             477
5,000,000+         752
50+                205
50,000+            479
50,000,000+        289
500+               330
500,000+           539
500,000,000+        72
Free                 1
dtype: int64


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [99]:
def installs_tonumber(serie):
    if (re.search('\d',serie)):
        serie = int(re.sub('\D', '', serie)) # Elimino tutti i caratteri esclusi i numerici
    return(serie)
        
google['Installs'] = google['Installs'].apply(installs_tonumber)
google['Installs'].head()

0       10000
1      500000
2     5000000
3    50000000
4      100000
Name: Installs, dtype: object

# 3- Trasformo 'Varies with device' in missing value

In [100]:
def missing(valore):
    if (valore == 'Varies with device'):
        valore = np.nan
    return(valore)

variabili = list(google.columns)
for variabile in variabili:
    google[variabile] = google[variabile].apply(missing)
    
google[variabile].head()

0    4.0.3 and up
1    4.0.3 and up
2    4.0.3 and up
3      4.2 and up
4      4.4 and up
Name: Android Ver, dtype: object

# 4a- trasforma in numero decimale 'android ver' 

In [101]:
#google.groupby('Android Ver').size()

vers_re = re.compile('^(?P<vers>\d\.\d\.{0,1}\d{0,1})')

def versione(v):
    if v is not np.nan:
        temp = v.split('-')
        if len(temp)>1: # In caso di trattino e quindi due versioni
            v = str(temp[1].strip()) # Rimuovo spazi iniziali e finali del secondo elemento della lista
        else: 
            v = vers_re.match(v).group('vers')
    return(v)

google['Android Ver'] = google['Android Ver'].apply(versione)
google['Android Ver'].head()

0    4.0.3
1    4.0.3
2    4.0.3
3      4.2
4      4.4
Name: Android Ver, dtype: object

# 4b) trasforma in numero decimale 'current ver'

In [102]:
google.groupby('Current Ver').size()

def versione2(ver):
    if ver is not np.nan:
        ver = re.sub('[^\d\.]', '', ver) # Elimino tutte le lettere e i caratteri che non sono punti
        if (len(ver)>1 and ver[0] == '.'): # Rimuovo il punto iniziale se presente
             ver = ver[1:]
        if ((ver == '') or (ver == '.')): # si presentano tali casi quando il valore originale è una parola
            return(np.nan)              
    return(ver)

google['Current Ver'] = google['Current Ver'].apply(versione2)
google['Current Ver'].head(10)


    

0       1.0.0
1       2.0.0
2       1.2.4
3         NaN
4         1.1
5         1.0
6         1.1
7    6.1.61.1
8       2.9.2
9         2.8
Name: Current Ver, dtype: object

# 5- Rimuovi i duplicati

In [103]:
google= google.drop(10472)
google['Reviews'] = google['Reviews'].astype(int)
duplicati= google.sort_values(by=['App','Reviews'],ascending=True)
duplicati_cut = duplicati[duplicati.duplicated(subset=['App'], keep='last')]
google = google.drop(duplicati_cut.index)

# 6- per ogni categoria, calcola number of apps

In [104]:
google.groupby('Category').size()

Category
ART_AND_DESIGN           61
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     222
BUSINESS                420
COMICS                   56
COMMUNICATION           315
DATING                  170
EDUCATION               105
ENTERTAINMENT            86
EVENTS                   64
FAMILY                 1878
FINANCE                 345
FOOD_AND_DRINK          112
GAME                    945
HEALTH_AND_FITNESS      288
HOUSE_AND_HOME           73
LIBRARIES_AND_DEMO       84
LIFESTYLE               369
MAPS_AND_NAVIGATION     131
MEDICAL                 395
NEWS_AND_MAGAZINES      254
PARENTING                60
PERSONALIZATION         376
PHOTOGRAPHY             281
PRODUCTIVITY            374
SHOPPING                202
SOCIAL                  239
SPORTS                  325
TOOLS                   829
TRAVEL_AND_LOCAL        219
VIDEO_PLAYERS           164
WEATHER                  79
dtype: int64

# 7- Per ogni categoria, calcola l'average rating

In [105]:
google.groupby('Category')['Rating'].mean()

Category
ART_AND_DESIGN         4.359322
AUTO_AND_VEHICLES      4.190411
BEAUTY                 4.278571
BOOKS_AND_REFERENCE    4.344970
BUSINESS               4.098479
COMICS                 4.181481
COMMUNICATION          4.121484
DATING                 3.980451
EDUCATION              4.349038
ENTERTAINMENT          4.129070
EVENTS                 4.435556
FAMILY                 4.184150
FINANCE                4.115563
FOOD_AND_DRINK         4.171277
GAME                   4.244432
HEALTH_AND_FITNESS     4.243033
HOUSE_AND_HOME         4.140984
LIBRARIES_AND_DEMO     4.178125
LIFESTYLE              4.093355
MAPS_AND_NAVIGATION    4.036441
MEDICAL                4.165862
NEWS_AND_MAGAZINES     4.121569
PARENTING              4.300000
PERSONALIZATION        4.332215
PHOTOGRAPHY            4.155894
PRODUCTIVITY           4.183389
SHOPPING               4.230556
SOCIAL                 4.247291
SPORTS                 4.216154
TOOLS                  4.040278
TRAVEL_AND_LOCAL       4.069519

# 8- Crea due dataframe: uno per i 'genres', uno che collega 'apps' e 'genres' 

In [106]:
print(google['Genres'].isnull().any()) # No missing

lista_app = []
lista_genres = []

for index,row in google.iterrows():
    l_generi = row['Genres'].split(';')
    for genre in l_generi:
        lista_genres.append(genre)
        lista_app.append(row['App'])

app_genre = pd.DataFrame({'App' : lista_app, 'Genres_one' : lista_genres})

False


In [107]:
df_genres = pd.DataFrame({'Genres' : list(set(lista_genres))})
df_genres.head()

,Genres
0,Word
1,Music
2,Personalization
3,Entertainment
4,Trivia


# 9- Per ogni 'genre' crea una nuova colonna nel dataframe originale; la nuova colonna avrà valore booleano (true se l'app ha un dato 'genre' associato)

In [108]:
l_false = [False] * google.shape[0] # Lista di False
lista_generi = list(set(lista_genres)) # Lista con generi univoci

# Inserisco in Playstore un attributo per ogni genere i cui valori sono tutti False
for genere in lista_generi:
    google[genere] = l_false 


# Setto a True tutti i valori degli attributi riguardanti il genere/generi dell'app
for index,row in google.iterrows():
    appoggio = row['Genres'].split(';')
    for genre in appoggio:
        google.at[index, genre.strip()] = True 
google

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Photography,Tools,Education,Parenting,Card,Productivity,Pretend Play,Casino,Comics,Art & Design
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,1.9e+07,10000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7e+06,5000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,2.5e+07,50000000,Free,0,Teen,Art & Design,...,False,False,False,False,False,False,False,False,False,True
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8e+06,100000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,True
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6e+06,50000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,1.9e+07,50000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,2.9e+07,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,3.3e+07,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1e+06,10000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,True
10,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,13880,2.8e+07,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True


# 10- Per ogni genere calcola l'average rating. Qual è il 'genre' con il rate più alto?

In [109]:
dff = pd.merge(app_genre, google, on='App')
dff.groupby('Genres_one')['Rating'].mean()

Genres_one
Action                     4.247697
Action & Adventure         4.288542
Adventure                  4.191026
Arcade                     4.277838
Art & Design               4.350000
Auto & Vehicles            4.190411
Beauty                     4.278571
Board                      4.291228
Books & Reference          4.343275
Brain Games                4.358065
Business                   4.098479
Card                       4.082609
Casino                     4.286486
Casual                     4.141014
Comics                     4.181481
Communication              4.121790
Creativity                 4.306250
Dating                     3.980451
Education                  4.290290
Educational                4.100000
Entertainment              4.093028
Events                     4.435556
Finance                    4.115563
Food & Drink               4.171277
Health & Fitness           4.243496
House & Home               4.140984
Libraries & Demo           4.178125
Lifestyle        

In [110]:
# Rating medio massimo
dff.groupby('Genres_one')['Rating'].mean().idxmax()

'Events'

# 11- Per ogni 'app' calcola l'income approssimato, ottenuto come prodotto tra 'number of installs' e 'price'

In [111]:
# Ispezione price
print(google.Price.isnull().any()) # No missing
google.groupby('Price').size()

False


Price
$0.99       145
$1.00         3
$1.04         1
$1.20         1
$1.26         1
$1.29         1
$1.49        46
$1.50         1
$1.59         1
$1.61         1
$1.70         2
$1.75         1
$1.76         1
$1.96         1
$1.97         1
$1.99        73
$10.00        2
$10.99        2
$109.99       1
$11.99        3
$12.99        3
$13.99        2
$14.00        1
$14.99        9
$15.46        1
$15.99        1
$154.99       1
$16.99        2
$17.99        2
$18.99        1
           ... 
$389.99       1
$39.99        2
$394.99       1
$399.99      12
$4.29         1
$4.49         9
$4.59         1
$4.60         1
$4.77         1
$4.80         1
$4.84         1
$4.85         1
$4.99        70
$400.00       1
$46.99        1
$5.00         1
$5.49         5
$5.99        26
$6.49         5
$6.99        10
$7.49         2
$7.99         7
$74.99        1
$79.99        1
$8.49         2
$8.99         5
$89.99        1
$9.00         1
$9.99        19
0          8905
Length: 92, dtype:

In [112]:
# Rimuovo eventuale valuta
google['Price']=google['Price'].apply(ins)
google
#google['Price'] = google['Price'].apply(lambda x: float(x.strip('$'))) 
#google[google['Price'] != 0]['Price'].head()


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Photography,Tools,Education,Parenting,Card,Productivity,Pretend Play,Casino,Comics,Art & Design
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,1.9e+07,10000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7e+06,5000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,2.5e+07,50000000,Free,0,Teen,Art & Design,...,False,False,False,False,False,False,False,False,False,True
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8e+06,100000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,True
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6e+06,50000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,1.9e+07,50000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,2.9e+07,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,3.3e+07,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1e+06,10000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,False,False,False,True
10,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,13880,2.8e+07,1000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,False,False,False,True


In [114]:
google['Income'] = google['Price'] * google['Installs']
google[google['Price'] != 0][['Price','Installs','Income']].head()

,Price,Installs,Income
290,499,100000,49900000
291,499,100000,49900000
427,399,100000,39900000
478,149,50,7450
479,299,100,29900


# 12- Per ogni 'app' calcola il minimo e il massimo di 'Sentiment Polarity'

In [117]:
sent_max = pd.DataFrame(playstore_review.groupby('App')['Sentiment_Polarity'].max()).reset_index()
sent_min = pd.DataFrame(playstore_review.groupby('App')['Sentiment_Polarity'].min()).reset_index()
sent = pd.merge(sent_max,sent_min, on='App')
sent = sent.rename(columns={'Sentiment_Polarity_x':'Sentiment_Polarity_max','Sentiment_Polarity_y':'Sentiment_Polarity_min'})
sent.head(10)

,App,Sentiment_Polarity_max,Sentiment_Polarity_min
0,10 Best Foods for You,1.000000,-0.800000
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000,-0.112500
2,11st,1.000000,-1.000000
3,1800 Contacts - Lens Store,0.838542,-0.300000
4,1LINE – One Line with One Touch,1.000000,-0.825000
5,2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,1.000000,-0.800000
6,21-Day Meditation Experience,0.587500,-0.265625
7,"2Date Dating App, Love and matching",1.000000,-0.645833
8,2GIS: directory & navigator,1.000000,-0.375000
9,2RedBeans,1.000000,-0.800000
